In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

print(encode('hello'))
print(decode(encode('hello')))

[61, 58, 65, 65, 68]
hello


In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])
print(data.shape)

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])
torch.Size([232311])


In [21]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ', x)
print('targets: ', y)

inputs:  tensor([[72, 58,  1, 62, 67, 72, 62, 57],
        [58, 57,  0, 50, 58, 55,  1, 68],
        [60, 60, 78,  1, 54, 67, 57,  1],
        [73, 61, 58,  1, 71, 68, 68, 59]], device='cuda:0')
targets:  tensor([[58,  1, 62, 67, 72, 62, 57, 58],
        [57,  0, 50, 58, 55,  1, 68, 59],
        [60, 78,  1, 54, 67, 57,  1, 55],
        [61, 58,  1, 71, 68, 68, 59, 24]], device='cuda:0')


In [22]:
'''
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('When Input is ', context, 'target is ', target) 

Output ->
When Input is  tensor([80]) target is  tensor(1)
When Input is  tensor([80,  1]) target is  tensor(1)
When Input is  tensor([80,  1,  1]) target is  tensor(28)
When Input is  tensor([80,  1,  1, 28]) target is  tensor(39)
When Input is  tensor([80,  1,  1, 28, 39]) target is  tensor(42)
When Input is  tensor([80,  1,  1, 28, 39, 42]) target is  tensor(39)
When Input is  tensor([80,  1,  1, 28, 39, 42, 39]) target is  tensor(44)
When Input is  tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is  tensor(32)
'''
print()

In [23]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        #print(f'logits[{index}]: {logits}')

        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        #index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            #print('probs: ', probs)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            #print('index_next: ', index_next)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
#print('Embedding_table: ', model.token_embedding_table)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
#print('encoded: ', m.generate(context, max_new_tokens=500))
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


VZ:p-F0ldE0pFvtd82_YVVl?z-n c&xJtPX;FTa[nd)]x3pJK﻿TYE"M7d1XU":w:pS3MVx[(SdYL]yA﻿9'A'oRYK7Dn[MO qrwPO&Byxs
)nDL
8S3Tin4(68[v7SwU-Q27pQgKr:h,RZ:fM5﻿MpvE"DF:NAd[CZ(88JsgVGH7&qi3)LLYb!,!AVT,)U[EerueJ_R_YY8UJ(EyiZJKrpj7:)4QKkZ-zWUN7Ljs[SF9ySFd﻿CwawTc1u&Y[Wg![a*LjX2_:0R;'q88N! ﻿[0n*O&cOmPjmOIxiDAw8Npi.q8EN!iG :WJ'vBo.?V15g_WN8 "Y﻿'5w;fIEsbAO2CR:RP[MEpe?ij﻿t2&QXWR:UR:o﻿MqcyqjIJdnpajoHoR8:3p"BBGJ!﻿j3RToabtgK;:qccvB[W:R4Ol]tb*YCfw
 chAB'e:_4Bw*wxFiGOW0jK7JW'DMV[fq[YBAYxJzv10Np-sF0X7)r"0q*itrGYY_Llz
-a]Z3


In [52]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.529, val loss: 2.568
step: 250, train loss: 2.536, val loss: 2.582
step: 500, train loss: 2.526, val loss: 2.583
step: 750, train loss: 2.545, val loss: 2.593
2.3267951011657715


In [53]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


TB-s!Mz. oioo w
CRE
he ts G"
teapas, w anere totr wnoutrJ(_n tas t mo thed ht.
Thed areene w me thar nt t s;!A(" less d at gg inthed y he "Y s.[Authey bun  anoworolaip. bong ts ug  l lu fo
indl ere d semarlil al.;pasethy S6H3Gck bR:j t
Fd;_jus tlede!y, elid inghed;douesug whavadim, simerse as busthid wemk she sechey Ocoen the cdstheskl
J﻿Nored t."AHRE"


owod mo rantontrcowingofOThen;0awhe m." andves o, m
"Buisin tony I'XBul.
dfowand.
" g gl, bus ak 'o?g rd t "sth istack't, "HCupu, y;ed bingad a
